In [ ]:
'''Code for feature extraction'''

In [ ]:
'''Importing preprocessed data(csv files) into DataFrame of pandas'''

import pandas as pd
df=pd.read_excel(r"C:\Users\Bhavya Prakash\OneDrive\Desktop\INMAS, DRDO\eeg-during-mental-arithmetic-tasks-1.0.0\pre_processed data (CSV files)\Subject_00_1.xlsx")
df

In [ ]:
'''Code for plotting of EEG channels for reference
    Plotting of PSDs
    Calculating theta/aplha bandpower'''

# Importing libraries
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.2)
import numpy as np
from scipy import signal
from scipy.integrate import simps


# Defining x-axis values that will be time
x=np.linspace(1,60,num=46592)
print(len(x))


# Defining y-axis values that will be volatges
y=df["EEG Fp1"]


# Defining sampling frequency as 256 Hz
sf = 256
# time = np.arange(df.size) / sf
time=x


# Plotting EEG signal from "Fp1" channel
fig, ax = plt.subplots(1, 1, figsize=(22, 8))
plt.plot(time, df["EEG Fp1"], lw=1.5, color='k')
plt.xlabel('Time (seconds)')
plt.ylabel('Voltage')
plt.xlim([time.min(), time.max()])
plt.title('EEG Fp1')
sns.despine()


# Plotting of Power Spectral Density
# Define window length (4 seconds)
win = 4 * sf
freqs, psd = signal.welch(df["EEG Fp1"], sf, nperseg=win)
sns.set(font_scale=1.2, style='white')
plt.figure(figsize=(8, 4))
plt.plot(freqs, psd, color='k', lw=2)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (V^2 / Hz)')
plt.ylim([0, psd.max() * 1.1])
plt.title("Welch's periodogram")
plt.xlim([0, freqs.max()])
sns.despine()


# Declaring of each wave(delta / theta / alpha / beta / gamma) in the PSD
# Declaring area of delta waves
low, high = 0.5, 4
# Find intersecting values in frequency vector
idx_delta = np.logical_and(freqs >= low, freqs <= high)
# Plot the power spectral density and fill the delta area
plt.figure(figsize=(7, 4))
plt.plot(freqs, psd, lw=2, color='k')
plt.fill_between(freqs, psd, where=idx_delta, color='skyblue')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (uV^2 / Hz)')
plt.xlim([0, 10])
plt.ylim([0, psd.max() * 1.1])
plt.title("Welch's periodogram for delta waves")
sns.despine()


# Declaring area of theta waves
low, high = 4,8
# Find intersecting values in frequency vector
idx_theta = np.logical_and(freqs >= low, freqs <= high)
# Plot the power spectral density and fill the delta area
plt.figure(figsize=(7, 4))
plt.plot(freqs, psd, lw=2, color='k')
plt.fill_between(freqs, psd, where=idx_theta, color='skyblue')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (uV^2 / Hz)')
plt.xlim([0, 10])
plt.ylim([0, psd.max() * 1.1])
plt.title("Welch's periodogram for theta waves")
sns.despine()


# Declaring area of alpha waves
low, high = 8,12
# Find intersecting values in frequency vector
idx_alpha = np.logical_and(freqs >= low, freqs <= high)
# Plot the power spectral density and fill the delta area
plt.figure(figsize=(7, 4))
plt.plot(freqs, psd, lw=2, color='k')
plt.fill_between(freqs, psd, where=idx_alpha, color='skyblue')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (uV^2 / Hz)')
plt.xlim([0, 16])
plt.ylim([0, psd.max() * 1.1])
plt.title("Welch's periodogram for alpha waves")
sns.despine()


# Declaring area of beta waves
low, high = 12,30
# Find intersecting values in frequency vector
idx_beta = np.logical_and(freqs >= low, freqs <= high)
# Plot the power spectral density and fill the delta area
plt.figure(figsize=(7, 4))
plt.plot(freqs, psd, lw=2, color='k')
plt.fill_between(freqs, psd, where=idx_beta, color='skyblue')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (uV^2 / Hz)')
plt.xlim([0, 34])
plt.ylim([0, psd.max() * 1.1])
plt.title("Welch's periodogram for beta waves")
sns.despine()


# Declaring area of gamma waves
low, high = 30,45
# Find intersecting values in frequency vector
idx_gamma = np.logical_and(freqs >= low, freqs <= high)
# Plot the power spectral density and fill the delta area
plt.figure(figsize=(7, 4))
plt.plot(freqs, psd, lw=2, color='k')
plt.fill_between(freqs, psd, where=idx_gamma, color='skyblue')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Power spectral density (uV^2 / Hz)')
plt.xlim([0, 50])
plt.ylim([0, psd.max() * 1.1])
plt.title("Welch's periodogram for gamma waves")
sns.despine()


# computing the absolute power of each wave
# Frequency resolution
freq_res = freqs[1] - freqs[0]  # = 1 / 4 = 0.25
# Compute the absolute power by approximating the area under the curve
delta_power = simps(psd[idx_delta], dx=freq_res)
print('Absolute delta power: %.12f uV^2' % delta_power)


# Computing realtive power of each wave
from scipy.integrate import simps
total_power = simps(psd, dx=freq_res)
delta_rel_power = delta_power / total_power
print('Relative delta power: %.3f' % delta_rel_power)


# Defining a function to calculate bandpower of each wave
def bandpower(data, sf, band, window_sec=None, relative=False):
    """Compute the average power of the signal x in a specific frequency band.

    Parameters
    ----------
    data : 1d-array
        Input signal in the time-domain.
    sf : float
        Sampling frequency of the data.
    band : list
        Lower and upper frequencies of the band of interest.
    window_sec : float
        Length of each window in seconds.
        If None, window_sec = (1 / min(band)) * 2
    relative : boolean
        If True, return the relative power (= divided by the total power of the signal).
        If False (default), return the absolute power.

    Return
    ------
    bp : float
        Absolute or relative band power.
    """
    from scipy.signal import welch
    from scipy.integrate import simps
    band = np.asarray(band)
    low, high = band
    # Define window length
    if window_sec is not None:
        nperseg = window_sec * sf
    else:
        nperseg = (2 / low) * sf

    # Compute the modified periodogram (Welch)
    freqs, psd = welch(data, sf, nperseg=nperseg)

    # Frequency resolution
    freq_res = freqs[1] - freqs[0]

    # Find closest indices of band in frequency vector
    idx_band = np.logical_and(freqs >= low, freqs <= high)

    # Integral approximation of the spectrum using Simpson's rule.
    bp = simps(psd[idx_band], dx=freq_res)

    if relative:
        bp /= simps(psd, dx=freq_res)
    return bp


# calculating theta/alpha bandpower
win_sec=60
db = bandpower(df["EEG Fp1"], sf, [4,8], win_sec) / bandpower(df["EEG Fp1"], sf, [8,12], win_sec)

# Delta/beta ratio based on the relative power
db_rel = bandpower(df["EEG Fp1"], sf, [4,8], win_sec, True) / bandpower(df["EEG Fp1"], sf, [8,12], win_sec, True)

print('Theta/alpha ratio (absolute): %.6f' % db)
print('Theta/alpha ratio (relative): %.6f' % db_rel)


In [ ]:
# importing data from saved CSV file consisting of PSDs of each channel of each subject
import pandas as pd


df1=pd.read_excel(r"C:\Users\Bhavya Prakash\OneDrive\Desktop\theta_alpha_ratio (WORKING STATE).xlsx")
print(df1)


# Calculate the average value of each row
row_averages = df1.mean(axis=1)

# Display the row averages
print(row_averages)
print(min(row_averages))
print(max(row_averages))


# distributing data to 5 arrays defining 5 mental workload levels i.e. 1,2,3,4,5.
arr1=[]
arr2=[]
arr3=[]
arr4=[]
arr5=[]

for i in row_averages:
    if 0.383703<=i<=0.7911246666666667:
        arr1.append(i)
    elif 0.7911246666666667<i<=1.1985463333333335:
        arr2.append(i)
    elif 1.1985463333333335<i<=1.6059680000000003:
        arr3.append(i)
    elif 1.6059680000000003<i<=2.013389666666667:
        arr4.append(i)
    elif 2.013389666666667<i<=2.4208113333333334:
        arr5.append(i)
    elif 2.4208113333333334<i<=2.8282330000000004:
        arr5.append(i)

print(len(arr1))
print(len(arr2))
print(len(arr3))
print(len(arr4))
print(len(arr5))

arr1.sort()
arr2.sort()
arr3.sort()
arr4.sort()
arr5.sort()


# importing data consisting of PSDs and Mental Workload levels.
df2=pd.read_excel(r"C:\Users\Bhavya Prakash\OneDrive\Desktop\theta_alpha_ratio (WORKING STATE).xlsx")
print(df2)


# Visualizing the data of a channel as an example
x=df2["EEG Fp1"][:25]
y=df2.Load[:25]
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
plt.scatter(x,y)
plt.xlabel("uV")
plt.ylabel("Load")
plt.title("EEG Fp1")
plt.show()